In [ ]:
import matplotlib

import numpy as np
import keras
import tensorflow as tf
import itertools    

from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU
from keras.layers import Conv2D, MaxPooling2D, Flatten, Input, Dense, Dropout, Reshape, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l1,l2

from keras.models import Model
from keras.callbacks import ModelCheckpoint

from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt

from load_data import load

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
keras.backend.set_session(sess)

In [ ]:
def autoencoder(X_train, X_test):
    input_layer = Input(shape=(48,48,3))
    
    encoder = Flatten()(input_layer)
    encoder = Dense(3456, activation='sigmoid')(encoder)
    encoder = Dense(1728, activation='sigmoid')(encoder)
    
    decoder = Dense(3456, activation='sigmoid')(encoder)
    decoder = Dense(48*48*3, activation='sigmoid')(decoder)
    decoder = Reshape((48,48,3))(decoder)

    autoencoder = Model(input_layer, decoder)
    opt = keras.optimizers.Adadelta()
    autoencoder.compile(optimizer=opt, loss='mean_squared_error')
    
    autoencoder.fit(X_train, X_train, epochs=200, batch_size=4, validation_split=0.2, callbacks=[ModelCheckpoint('./potato', save_best_only=False, period=10)])
    print(autoencoder.evaluate(X_test, X_test))
    encoder = Model(input_layer, encoder)
    return autoencoder, encoder, input_layer

In [ ]:
def downsample(X):
    import Image
    resized = []
    for x in X:
        img = Image.fromarray(np.uint8(x * 255))
        resized.append(np.asarray(img.resize((48, 48), Image.NEAREST)).astype('float32')/255.0)
    return np.array(resized)

In [ ]:
def reconstruct_image(vae, images, index=0):
    x_test = vae.predict(images)
    x_test = np.uint8(x_test * 255)
    plot_image = np.concatenate((np.uint8(images[index]*255), x_test[index]), axis=1)
    imgplot = plt.imshow(plot_image)

In [ ]:
(X_train, Y_train), (X_test, Y_test), cross_val_indices = load()
X_train = downsample(X_train) 
X_test = downsample(X_test)
aec, enc, inp = autoencoder(X_train, X_test)

aec.save("part1_aec")
enc.save("part1_enc")

In [ ]:
reconstruct_image(aec, X_test, 0)
plt.show()

In [ ]:
def make_model_from_enc(inp, enc):
    encoder = Dense(864, activation='sigmoid')(enc)
    encoder = Dense(432, activation='sigmoid')(encoder)
    encoder = Dense(10, activation='softmax')(encoder)
    whole_model = Model(inp, encoder)
    opt = keras.optimizers.Adadelta()
    
    whole_model.compile(optimizer=opt, loss='categorical_crossentropy')
    whole_model.fit(X_train, Y_train, epochs=100, batch_size=4, validation_split=0.2, callbacks=[ModelCheckpoint('./together', save_best_only=False, period=10)])
    print(whole_model.evaluate(X_test, Y_test))
    return whole_model

In [ ]:
final_model = make_model_from_enc(inp, enc.output)

In [ ]:
def plot_classwise(y_test, y_pred, name):
    y_test_aho = np.argmax(y_test, axis=1)
    y_pred_aho = np.argmax(y_pred, axis=1)
    print(classification_report(y_test_aho, y_pred_aho))

In [ ]:
plot_classwise(Y_test, final_model.predict(X_test), 'vanilla autoencoder')